In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files

In [ ]:
path = "/content/drive/MyDrive/"

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.naive_bayes import MultinomialNB
import nltk # it is the one of most comman libraries for Natural Language Process
import re # Regular Expression library

In [ ]:
with open(path + "gender-classifier-data.csv", encoding="latin1", errors='ignore') as f:
  twitter_data = pd.read_csv(f)

In [ ]:
#Use only the gender and text column
data = pd.concat([twitter_data.gender, twitter_data.text], axis=1)

In [ ]:
#Drop Brand and unknown
data = data[data["gender"] != "brand"]
data = data[data["gender"] != "unknown"]

In [ ]:
import nltk # natural language took kit
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
# genders have two options (male/female). so i changed male to 0 and female to 1.
data.gender = [1 if gender == "female" else 0 for gender in data.gender]

In [ ]:
import nltk
description_list = []
for description in data.text:
    description = re.sub("[^a-zA-Z]", " ", description)
    description = description.lower()
    description = nltk.word_tokenize(description)
    lemma = nltk.WordNetLemmatizer()
    description = [lemma.lemmatize(word) for word in description]
    description = " ".join(description)
    description_list.append(description)

In [ ]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
max_features = 5000
ti = TfidfVectorizer(stop_words = "english", ngram_range=(1, 2))
sparce_matrix_ti = ti.fit_transform(description_list).toarray()
print("top used {} words: {}".format(max_features, ti.get_feature_names()))

top used 5000 words: ['aaa', 'aaaaa', 'aaaaa smile', 'aaaaaaaa', 'aaaaaaaa fun', 'aaaaaaall', 'aaaaaaall weekend', 'aaaaahhhhhmmmm', 'aaaaahhhhhmmmm driving', 'aaahovu', 'aaallll', 'aaallll scratched', 'aaand', 'aaand got', 'aafegt', 'aahhwza', 'aaliylbcoi', 'aam', 'aam gilbert', 'aap', 'aap alka', 'aap character', 'aapl', 'aapl huge', 'aaron', 'aaron cook', 'aaron gt', 'aaron stopped', 'aaroncarpenter', 'aaroncarpenter roll', 'aarondmiller', 'aarondmiller actforisrael', 'aaroneaves', 'aaroneaves know', 'aaronklemz', 'aaronklemz radio', 'aasghbz', 'aatwband', 'aatwband listening', 'aawnc', 'aawnc arguing', 'ab', 'ab bear', 'ab fab', 'ab highschool', 'ab http', 'ab just', 'ab kettle', 'ab nrdcfood', 'ab soul', 'ab workout', 'abacus', 'abacus math', 'abandon', 'abandon friend', 'abandoned', 'abandoned building', 'abandoned dispossessed', 'abb', 'abb bass', 'abba', 'abba http', 'abba just', 'abbas', 'abbas da', 'abbas hashi', 'abbey', 'abbey changed', 'abbeykids', 'abbeykids reimerandruby

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# get column 0, which is the gender as an NDARRAY of shape ( n,) 
y = data.iloc[:, 0].values
x = sparce_matrix_ti

In [ ]:
# Feature extraction
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif,f_classif
feature_selector = SelectKBest(score_func=chi2, k=1000).fit(sparce_matrix_ti, y)
# get best feature names
important_feature_names = feature_selector.get_feature_names_out(ti.get_feature_names_out())
name_and_scores = list(zip(important_feature_names, feature_selector.scores_))
name_and_scores.sort(key=lambda x: x[1], reverse=True)
#for ns in name_and_scores:
#  print(ns)

In [ ]:
# TF-IDF and SVM 
# cross validation, use 20% of the data to test;42 is a common choice to decide the seed of randomness
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(feature_selector.transform(sparce_matrix_ti), y, test_size = 0.2, random_state = 42)
print(y_test)

[0 0 1 ... 0 1 1]


In [ ]:
#Import svm model
from sklearn import svm
from sklearn import metrics

#Classifier
svm_classifier = svm.SVC(kernel='linear') # Linear Kernel

#Train
svm_classifier.fit(x_train, y_train)

#Predict
y_pred = svm_classifier.predict(x_test)

# SVM accuacy
print("SVM Accuracy tf-idf text:",metrics.accuracy_score(y_test, y_pred))

SVM Accuracy tf-idf text: 0.6271642939592151


In [ ]:
# now, we will implement "bag of words" method

from sklearn.feature_extraction.text import CountVectorizer

max_features = 5000 # we will take top 5000 feature,means it will give 5000 most common words 

cv = CountVectorizer(max_features=max_features, stop_words = "english")
sparce_matrix = cv.fit_transform(description_list).toarray()

print("top used {} words: {}".format(max_features, cv.get_feature_names()))

top used 5000 words: ['ab', 'abc', 'ability', 'able', 'abortion', 'absolute', 'absolutely', 'abt', 'abuja', 'abuse', 'ac', 'acc', 'accept', 'acceptable', 'accepted', 'access', 'accident', 'accidentally', 'accomplishment', 'according', 'account', 'accurate', 'accused', 'ace', 'ache', 'acoustic', 'act', 'acting', 'action', 'active', 'activist', 'activity', 'actor', 'actress', 'actual', 'actually', 'ad', 'adam', 'add', 'added', 'addition', 'address', 'adele', 'admiration', 'admire', 'admit', 'adonissuperman', 'adorable', 'adult', 'advance', 'advantage', 'adventure', 'advice', 'aesthetic', 'af', 'affair', 'affect', 'affected', 'afford', 'afghanistan', 'afifest', 'afraid', 'africa', 'african', 'afternoon', 'age', 'agency', 'agenda', 'agent', 'ago', 'agree', 'agreed', 'agreement', 'ah', 'ahahaha', 'ahead', 'ahh', 'aid', 'ain', 'aint', 'air', 'airport', 'aka', 'al', 'alan', 'alarm', 'album', 'alcohol', 'aldubpredictions', 'alert', 'alex', 'ali', 'alien', 'alive', 'allah', 'allen', 'allergy', 

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# Feature extraction
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif,f_classif
feature_selector = SelectKBest(score_func=chi2, k=1000).fit(sparce_matrix, y)
# get best feature names
important_feature_names = feature_selector.get_feature_names_out(cv.get_feature_names_out())
name_and_scores = list(zip(important_feature_names, feature_selector.scores_))
name_and_scores.sort(key=lambda x: x[1], reverse=True)
#for ns in name_and_scores:
#  print(ns)

In [ ]:
# get column 0, which is the gender as an NDARRAY of shape ( n,) 
y = data.iloc[:, 0].values
x = sparce_matrix
print(sparce_matrix)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
# cross validation, use 10% of the data to test;42 is a common choice to decide the seed of randomness
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(feature_selector.transform(sparce_matrix), y, test_size = 0.2, random_state = 42)
print(y_test)

[0 0 1 ... 0 1 1]


In [ ]:
#Import svm model
from sklearn import svm
from sklearn import metrics

#Classifier
svm_classifier = svm.SVC(kernel='linear') # Linear Kernel

#Train
svm_classifier.fit(x_train, y_train)

#Predict
y_pred = svm_classifier.predict(x_test)

# SVM accuacy
print("SVM Accuracy count vector text:",metrics.accuracy_score(y_test, y_pred))

SVM Accuracy count vector text: 0.6590996537129665
